In [ ]:
#to make ceiling video.....

#get asterism positions at ONE MOMENT (at kaifeng), use this as BASE

#PLOT, for each time:
    #visible asterisms at alpha --> use asterism_visibility and twi_percents

#then, animate sun/moon motion on top, separately.....(using sun moon motion video as a guide)
    #probably in after effects

In [38]:
#load twi/vis info
    
with open("DATA/asterism_visibility.pickle", "rb") as f:
    vis_info = pickle.load(f)

with open("DATA/twi_percents.json", "r") as f:
    twi_percents = json.load(f)
    
with open("DATA/sun_updown.json", "r") as f:
    sun_pos = json.load(f)
    
with open("DATA/moon_updown.json", "r") as f:
    moon_pos = json.load(f)

In [1]:
def setup():
    
    #load info
    eph = load('de421.bsp') 
    with load.open(hipparcos.URL) as f:
        all_stars = hipparcos.load_dataframe(f)
        
    #create sun, earth, and moon 
    sun = eph['sun']
    earth = eph['earth']
    moon = eph['moon']
    
    #create timescale
    ts = load.timescale()
    
    return eph, all_stars, sun, earth, moon, ts






def set_location(lat, latD, lon, lonD, elevation_m):
    #create location
    
    location = earth + wgs84.latlon(lat * latD, lon * lonD, elevation_m=elevation_m)
    
    return location






def make_star(hipp_num, all_stars):
    
    star = Star.from_dataframe(all_stars.loc[hipp_num])
    
    return star











# def obj_over_time(obj, location, times):
    
#     #get list of alt, az, and dist of object at location during all times listed ts.utc()
#     alts = []
#     azs = []
#     dists = []
    
#     for time in times:
#         alt, az, dist = get_object_info(obj, location, time)
#         alts.append(alt)
#         azs.append(az)
#         dists.append(dist)
        
#     return alts, azs, dists



def altaz_to_polar(alt, az):
    
    #stereographic projection
    
    if alt >= 0:
        r = np.sin(alt + np.pi/2) / (1 - np.cos(alt + np.pi/2))
    else:
        r = -(alt) / (np.pi/2) + 1
         
    theta = az
    
    return r, theta


def make_constellation(all_stars, hipps):
    
    const = []
    for hip in hipps:
        #print(hip)
        star = make_star(hip, all_stars)
        
        const.append(star)
        #print(star)
        
    return const
    


def get_object_info(obj, location, time):
    
    #get alt, az, and dist of specified object at particular time & place 
    #time as ts.utc()
    
    obj_observe = location.at(time).observe(obj)
    obj_app = obj_observe.apparent()
    
    alt, az, dist = obj_app.altaz()
    
    return alt, az, dist

    
def get_constellations(all_stars, const_hipps):
    
    #make stars for all constellations and put into dictionary based on const name 
    
    consts = {}
    for key, value in const_hipps.items():
        consts[key] = make_constellation(all_stars, value)
    
    return consts




def get_all_constellations_info(const, location, time): 
    
    #get altaz & dist for each star in each constellation, at ONE timeslice 
    #save as "constellation": [[alt, az, dist], [alt, az, dist], ...... ]
    
    const_info = {}
    for star_name, stars in const.items():
        #print("get all", const[star_name])
        const_info[star_name] = get_constellation_info(stars, location, time)
    
    
    return const_info 

def get_constellation_info(star_list, location, time):
    
    #get list of [[alt, az, dist], [alt, az, dist], ...] for ONE constellation at ONE timeslice
    all_info = []
    for star in star_list:
        #print("get const", star)
        info = get_object_info(star, location, time)
        all_info.append(info)
    
    return all_info

def get_info_slice(location, time, sun, moon, constellations):
    
    #get info on sun, moon, and all constellations at ONE time 
    sun_info = get_object_info(sun, location, time)
    moon_info = get_object_info(moon, location, time)
    const_info = get_all_constellations_info(constellations, location, time)
    
    return sun_info, moon_info, const_info #sun and moon as [alt, az, dist], const as dictionary
    
def get_all_rtheta(loc, times, sun, moon, constellations): #get sun, moon, const rtheta info 
                                                            #at location at times
    
    all_sun_info = []
    all_moon_info = []
    all_const_info = []

    for time in times:
        sun_info, moon_info, const_info = get_info_slice(loc, time, sun, moon, constellations)
        all_sun_info.append(sun_info)
        all_moon_info.append(moon_info)
        all_const_info.append(const_info)
        
    #convert altaz to r, theta 
    sun_rs = []
    sun_thetas = []
    moon_rs = []
    moon_thetas = []
    const_rs = {}
    const_thetas = {}

    for sun_info in all_sun_info:
        alt, az = sun_info[0:2]
        r, theta = altaz_to_polar(alt.radians, az.radians)
        sun_rs.append(r)
        sun_thetas.append(theta)


    for moon_info in all_moon_info:
        alt, az = moon_info[0:2]
        r, theta = altaz_to_polar(alt.radians, az.radians)
        moon_rs.append(r)
        moon_thetas.append(theta)


    for name, value in all_const_info[0].items():
        num_stars = len(value)
        const_rs[name] = []
        const_thetas[name] = []
        for i in range(num_stars):
            const_rs[name].append([])
            const_thetas[name].append([])

    #print(const_rs)

    for const_info in all_const_info:
        for name, stars in const_info.items():
            for i in range(len(stars)):
                star = stars[i]
                alt, az = star[0:2]
                r, theta = altaz_to_polar(alt.radians, az.radians)
                const_rs[name][i].append(r)
                const_thetas[name][i].append(theta)
                
    return sun_rs, sun_thetas, moon_rs, moon_thetas, const_rs, const_thetas    

In [2]:
from skyfield.api import Star, load, N,S,E,W, wgs84
from skyfield.data import hipparcos
from skyfield import almanac

import matplotlib.pyplot as plt
import numpy as np

In [3]:
#load constellation numberes
import json

with open('DATA/constellation_hipnums.json') as f:
    constellation_hipps = json.load(f)

In [4]:
#SETUP

eph, all_stars, sun, earth, moon, ts = setup() #load setup stuff

kaifeng = set_location(34.795, N, 114.345, E, 75) #set locations
northpole = set_location(90, N, 135, W, 147)

constellations = get_constellations(all_stars, constellation_hipps) #make all constellations

In [5]:
#load times
import pickle
with open("DATA/times.pickle", "rb") as f:
    times = pickle.load(f)

In [6]:
#get constellation info at NORTH POLE, at start and end 
sun_r, sun_t, moon_r, moon_t, const_r, const_t = get_all_rtheta(northpole, [times[0]], 
                                                                sun, moon, constellations)

In [7]:
const_r

{'P-7-northpole': [[0.15991538533961183],
  [0.13921158456029475],
  [0.12553859210041154],
  [0.1091627185940693],
  [0.05758133033064293]],
 'P-6-fourad': [[0.05028090916669368],
  [0.06510808238199542],
  [0.07590695949222893]],
 'P-5-curvedarray': [[0.10695681551247303],
  [0.06961867134286573],
  [0.02979988148044392],
  [0.006362693321734481],
  [0.032599195333428664],
  [0.04929873653300441]],
 'P-4-greatemp': [[0.03390137592096678]],
 'P-15-celestpill': [[0.0652116083948665],
  [0.10392507335625935],
  [0.11847549198465847],
  [0.11779136257531157],
  [0.08747335086357935]],
 'P-13-maids': [[0.14624737202458965],
  [0.12777771818947153],
  [0.13852004834747217]],
 'P-17-sixjia': [[0.09134187877663988],
  [0.1141637798537208],
  [0.09211525751047692],
  [0.06635804239603692],
  [0.0769841072252009],
  [0.09424747037607849]],
 'P-14-offarchs': [[0.16429840883162058]],
 'P-12-fivesecr': [[0.19315948763454274],
  [0.18745695378299496],
  [0.22547990642073734],
  [0.1843782484360437

In [8]:
#save const info for ceiling video generation

with open("CEILING_VIDEO/const_r.json", "w") as f:
    json.dump(const_r, f)

with open("CEILING_VIDEO/const_t.json", "w") as f:
    json.dump(const_t, f)

In [39]:
sr = [sun_r[0], sun_r[-1]]
st = [sun_t[0], sun_t[-1]]
mr = [moon_r[0], moon_r[-1]]
mt = [moon_t[0], moon_t[-1]]

vis = [vis_info[0], vis_info[-1]]

twi = [twi_percents[0], twi_percents[-1]]

spos = [sun_pos[0], sun_pos[-1]]
mpos = [moon_pos[0], moon_pos[-1]]

In [36]:
sr

[1.1117460935588546, 1.1011756079704094]

In [44]:
plot_at_slices(sr, st, mr, mt, const_r, const_t, vis,
              spos, mpos, twi_percents, [1], 2, "forceiling", save=True)

<Figure size 720x720 with 0 Axes>

In [34]:
def plot_at_slices(sun_r, sun_t, moon_r, moon_t, const_r, const_t, vis_info, 
                   sun_pos, moon_pos, twi_percents, slices, offset, filename, save=False):
    
    for hour in slices:

        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': 'polar'})
        for name, rs in const_r.items():
            thetas = const_t[name]

            thisconst_rs = []
            thisconst_thetas = []

            for i in range(len(thetas)):
                star_rs = rs[i]
                star_theta = thetas[i]
                thisconst_rs.append(star_rs[hour])
                thisconst_thetas.append(star_theta[hour])
                
            
            #first, plot everything in black
            ax.plot(thisconst_thetas, thisconst_rs, "o-", color='k', 
                    markersize=2, linewidth=1, alpha = 0.25)
            
            #then, plot in color IF it's visible (for now, if one is visible, the whole const is visible)
            visibility = vis_info[hour][name]
            
            if visibility:
                a = twi_percents[hour]
                if name[0] == "P":
                    ax.plot(thisconst_thetas, thisconst_rs, "o-", color='purple', 
                            markersize=2, linewidth=1, alpha = a)

                elif name[0] == "R":            
                    ax.plot(thisconst_thetas, thisconst_rs, "o-", color='red', 
                            markersize=2, linewidth=1,  alpha = a)

                elif name[0] == "B":            
                    ax.plot(thisconst_thetas, thisconst_rs, "o-", color='blue', 
                            markersize=2, linewidth=1,  alpha = a)

                elif name[0] == "K":            
                    ax.plot(thisconst_thetas, thisconst_rs, "o-", color='green', 
                            markersize=2, linewidth=1,  alpha = a)

                elif name[0] == "W":            
                    ax.plot(thisconst_thetas, thisconst_rs, "o-", color='orange', 
                            markersize=2, linewidth=1,  alpha = a)

        
        #if sun is visible, plot alpha = 1
        #else, plot alpha = 0.25
        if sun_pos[hour]:
            ax.scatter(sun_t[hour], sun_r[hour], color="yellow", s=300, zorder=5, alpha=1)
        else:
            ax.scatter(sun_t[hour], sun_r[hour], color="yellow", s=300, zorder=5, alpha=0.25)
            
        if moon_pos[hour]:
            ax.scatter(moon_t[hour], moon_r[hour], color='grey', s=300, zorder=5, alpha=1)
        else:
            ax.scatter(moon_t[hour], moon_r[hour], color='grey', s=300, zorder=5, alpha=0.25)
        
        for i in range(1, 8):
            
            if (i<7):
                ax.plot(np.linspace(0, 2*np.pi, 100), np.ones(100)*i*0.25, 
                        color='grey', linestyle='-', zorder=0, linewidth=1, alpha=0.2)
            elif i == 7:
                ax.plot(np.linspace(0, 2*np.pi, 100), np.ones(100)*i*0.25, 
                        color='k', linestyle='-', zorder=0, linewidth=2, alpha=1)

        ax.set_rmax(1.75)
        ax.set_theta_offset((10-hour) * np.pi/48 + np.pi/2 + offset) #lol
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax.grid(visible=False)
        #ax.grid(visible=True, axis="y")
        
        if save == False:
            plt.show()
        else:
            pltname = str(hour) + filename + ".png"
            plt.savefig(pltname, transparent=True, dpi=200, bbox_inches="tight")
            plt.clf()

In [15]:
def get_all_const_info(loc, times, constellations):
    
    #create const_r and const_t
    #const_r[NAME][0][2] --> for NAME, 0th star, 2nd timeslice
    
    const_r = {}
    const_t = {}
    
    #build dictionaries first
    for name, starlist in constellations.items():
        const_r[name] = [[] for x in starlist]
        const_t[name] = [[] for x in starlist]
        
    for i in range(len(times)):
        time = times[i]
        for name, starlist in constellations.items(): 
            